In [1]:
# You better run this code on GG Colab
!git clone https://github.com/KCDichDaNgu/KC4.0_MultilingualNMT.git
!git clone https://github.com/pytorch/fairseq.git
!pip install fairseq
!pip install sacremoses
!pip install sentencepiece


Cloning into 'KC4.0_MultilingualNMT'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 268 (delta 5), reused 3 (delta 3), pack-reused 255
Receiving objects: 100% (268/268), 45.75 MiB | 8.85 MiB/s, done.
Resolving deltas: 100% (68/68), done.
Cloning into 'fairseq'...
remote: Enumerating objects: 34558, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 34558 (delta 5), reused 10 (delta 4), pack-reused 34543
Receiving objects: 100% (34558/34558), 24.08 MiB | 22.25 MiB/s, done.
Resolving deltas: 100% (25120/25120), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 63.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
 

# Data Preprocessing


In [2]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 70.8 MB/s eta 0:00:00


In [3]:
# !rm -rf /content/KC4.0_MultilingualNMT

In [4]:
from pyvi import ViTokenizer
import os

In [5]:
test_dir = "/content/KC4.0_MultilingualNMT/data/iwslt_en_vi"
def load_data(source_file):
    source_sents = open(source_file, "r").readlines()
    source_data = []

    for sentence in source_sents:
        source_data.append(sentence.strip())
    return source_data

def token_file(path):
    src_data = load_data(path)
    src_data[0:10]
    token_sentence = [ViTokenizer.tokenize(i).split() for i in src_data]
    # token_sentence[0:2]
    token_sentence = [" ".join(words) for words in token_sentence]
    # token_sentence[0:4]
    with open(path, "w") as writefile:
        for line in token_sentence:
            # print(line, file=writefile)
            writefile.write(line)
            writefile.write('\n')


In [6]:
files = ['train.vi', "tst2012.vi", "tst2013.vi"]
for file in files:
  token_file(f"{test_dir}/{file}")

# Run Fairseq

In [7]:
!cp /content/prepare-iwslt14.sh /content/fairseq/examples/translation/prepare-iwslt14.sh

In [8]:
%cd /content
%cd fairseq/examples/translation
!bash prepare-iwslt14.sh
%cd ../..

/content
/content/fairseq/examples/translation
Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148097, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 148097 (delta 323), reused 441 (delta 292), pack-reused 147572
Receiving objects: 100% (148097/148097), 129.88 MiB | 17.72 MiB/s, done.
Resolving deltas: 100% (114349/114349), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 597 (delta 8), reused 12 (delta 4), pack-reused 576
Receiving objects: 100% (597/597), 252.23 KiB | 1.68 MiB/s, done.
Resolving deltas: 100% (357/357), done.
pre-processing train data...
Tokenizer Version 1.1
Language: en
Number of threads: 8

Tokenizer Version 1.1
Language: vi
Numbe

In [9]:
# ! rm -Rf ./fairseq

# %cd .
# !mv /content/prepare-iwslt14.sh /content/fairseq/examples/translation/prepare-iwslt14.sh

In [10]:
import subprocess

# Set the command
command = [
    "fairseq-preprocess",
    "--source-lang", "en",
    "--target-lang", "vi",
    "--trainpref", "examples/translation/iwslt.tokenized.en-vi/train",
    "--validpref", "examples/translation/iwslt.tokenized.en-vi/valid",
    "--testpref", "examples/translation/iwslt.tokenized.en-vi/test",
    "--destdir", "data-bin/iwslt.tokenized.en-vi",
    "--workers", "20"
]

# Execute the command
subprocess.run(command, check=True)


CompletedProcess(args=['fairseq-preprocess', '--source-lang', 'en', '--target-lang', 'vi', '--trainpref', 'examples/translation/iwslt.tokenized.en-vi/train', '--validpref', 'examples/translation/iwslt.tokenized.en-vi/valid', '--testpref', 'examples/translation/iwslt.tokenized.en-vi/test', '--destdir', 'data-bin/iwslt.tokenized.en-vi', '--workers', '20'], returncode=0)

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 fairseq-train \
#     data-bin/iwslt.tokenized.en-vi \
#     --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
#     --optimizer adam --adam-betas "(0.9, 0.98)" --clip-norm 0.0 \
#     --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
#     --dropout 0.3 --weight-decay 0.0001 \
#     --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
#     --max-tokens 4096 \
#     --eval-bleu \
#     --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
#     --eval-bleu-detok moses \
#     --eval-bleu-remove-bpe \
#     --eval-bleu-print-samples \
#     --best-checkpoint-metric bleu --maximize-best-checkpoint-metric

!mkdir -p checkpoints/fconv_wmt_en_de
!fairseq-train \
    data-bin/iwslt.tokenized.en-vi \
    --arch fconv_wmt_en_de \
    --dropout 0.2 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer nag --clip-norm 0.1 \
    --lr 0.5 --lr-scheduler fixed --force-anneal 50 \
    --max-tokens 4000 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir checkpoints/fconv_wmt_en_de



2023-05-15 15:54:33.823914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 15:54:34.683274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-15 15:54:35 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2023-05-15 15:54:36 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-05-15 15:54:39 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': 